In [ ]:
import time
import datetime
from scipy import stats
import pandas as pd
import numpy as np
from plotly import tools
import plotly.plotly as py
import plotly.graph_objs as go
from plotly.offline import download_plotlyjs, init_notebook_mode, iplot
init_notebook_mode(connected=True)

In [ ]:
def calc_returns(dist, sharpe, vol):
    drift = sharpe*vol/252
    return np.cumsum([i*vol + drift for i in dist])

def gen_t_dist(n, s=42, d=4):
    np.random.seed(s)
    return stats.t.rvs(d, size=n)

def renorm(dist, new_s=252**0.5):
    m = np.mean(dist)
    s = np.std(dist)
    return [(i-m)/(s*new_s) for i in dist]

def cumulative_max(s):
    m = 0.
    cummax = []
    for c in s:
        if c > m:
            cummax.append(c)
            m = c
        else:
            cummax.append(m)
    return cummax

def calc_apriori(dd, sh, v):
    apriori = []
    for t,d in enumerate(dd):
        t  = (t+1)/252.
        m = sh*v*t
        s = (t**0.5)*v
        p = stats.norm(m, s).cdf(d)
        d_text = 'Dd: ' + str(round(d, 3)*100) + '%'
        p_text = 'Apriori: ' + str(round(p, 3)*100) + '%'
        aprior.append(d_text + '<br>' + p_text)
    return apriori

def worst_per_month(s):
    return [min(s[i*21:(i+1)*21]) for i in range(len(s)/21)]

def worst_per_year(s):
    return [min(s[i * 252:(i + 1) * 252]) for i in range(len(s) / 252)]

In [ ]:
n_seeds = 5
n_years = 100
n_days = n_years * 252
sharpe = 1
vol = .1

daily_returns = renorm(gen_t_dist(n_days, s=0))
cumul_returns = calc_returns(daily_returns, sharpe, vol)
ddown = cumul_returns - cumulative_max(cumul_returns)

In [ ]:
#np.save('t_stat_samples', student_dist)
"""
for sh in range(10):
    sharpe = (sh+1)/10.
    vol = .1
    cr2 = calc_returns(cr, sharpe, vol)

    ddown = cr2 - cumulative_max(cr2)
    min_drawdowns = worst_per_year(ddown)
    x_trace = sorted(min_drawdowns)

    y_trace = np.cumsum([1./len(x_trace)]*len(x_trace))
    x2 = np.interp(0.9995, y_trace, x_trace[::-1])
    print(sharpe, x2)
"""

In [ ]:
data = [go.Histogram(x=worst_per_year(ddown))]
layout = dict(xaxis=dict(tickformat='.1%', 
                         autorange='reversed'))
fig = dict(data=data, layout=layout)
iplot(fig)

In [ ]:
x_trace=sorted(min_drawdowns)
y_trace = np.cumsum([1./len(x_trace)]*len(x_trace))
x1 = np.interp(0.5, y_trace, x_trace[::-1])
x2 = np.interp(0.95, y_trace, x_trace[::-1])

In [ ]:
data = [go.Scatter(y=ddown, 
                   name='Drawdown',
                   text = prob,
                   hoverinfo = 'text',
                   showlegend=False), 
        go.Scatter(x=[0,len(ddown)],
                   y=[x2, x2],
                   mode='lines',
                   name='95%: ' + str(round(x2, 3)*100) + '%')]
layout = dict(xaxis=dict(range=[0,len(ddown)]),
              yaxis=dict(range=[min(ddown), 0],
                         tickformat='.1%'))
fig=dict(data=data, layout=layout)
iplot(fig)

In [ ]:
cdf_data = go.Scatter(x=x_trace,
                      y=y_trace[::-1],
                      mode = 'lines',
                      name='CDF')
y_inter1 = go.Scatter(x=[0, x1],
                      y=[0.5, 0.5],
                      mode = 'lines',
                      hoverinfo='none',
                      line = dict(color = ('rgb(0,0,0)'),
                                  width = 1,), )
x_inter1 = go.Scatter(x=[x1, x1],
                      y=[0, 0.5],
                      mode = 'lines',
                      hoverinfo='none',
                      line = dict(color = ('rgb(0,0,0)'),
                                  width = 1,), )
y_inter2 = go.Scatter(x=[0, x2],
                      y=[0.95, 0.95],
                      mode = 'lines',
                      hoverinfo='none',
                      line = dict(color = ('rgb(0,0,0)'),
                                  width = 1,), )
x_inter2 = go.Scatter(x=[x2, x2],
                      y=[0, 0.95],
                      mode = 'lines',
                      hoverinfo='none',
                      line = dict(color = ('rgb(0,0,0)'),
                                  width = 1,), )
data =[cdf_data, y_inter1, x_inter1, y_inter2, x_inter2]
layout=dict(
    xaxis=dict(range=[x_trace[-1], x_trace[0]]),
    yaxis=dict(range=[0,1]),
    showlegend=False,
    annotations=[
        dict(
            x=x1,
            y=0,
            xref='x',
            yref='y',
            text='50%: ' + str(round(x1, 3)),
            showarrow=True,
            font=dict(

                size=14,
                color='#ffffff'
            ),
            align='center',
            arrowhead=2,
            arrowsize=1,
            arrowwidth=2,
            arrowcolor='#7C878E',
            ax=60,
            ay=-50,
            bordercolor='#7C878E',
            borderwidth=2,
            borderpad=4,
            bgcolor='rgb(78, 135, 160)',
            opacity=0.8
        )
    ,
        dict(
            x=x2,
            y=0,
            xref='x',
            yref='y',
            text='95%: ' + str(round(x2, 3)),
            showarrow=True,
            font=dict(

                size=14,
                color='#ffffff'
            ),
            align='center',
            arrowhead=2,
            arrowsize=1,
            arrowwidth=2,
            arrowcolor='#7C878E',
            ax=60,
            ay=-50,
            bordercolor='#7C878E',
            borderwidth=2,
            borderpad=4,
            bgcolor='rgb(78, 135, 160)',
            opacity=0.8
        )
    ]
)
fig = dict(data=data, layout=layout)
iplot(fig)